## Tiger State Geometry Fetch for North Carolina

In [1]:
import os,sys;
from arcgis.gis import GIS;
from arcgis.geocoding import Geocoder, batch_geocode;
from arcgis.geometry import Point, Geometry, distance;
from arcgis.mapping import MapServiceLayer;
import arcpy;
import pandas as pd;

base = r'C:\Users\PDziemiela\Documents\GitHub\Jupyter_Notebooks\CANC';


In [2]:
g_purge = False;

geopkg = base + os.sep + 'CANC.gpkg';

def add_fields(target,farray):

    for item in farray:
        
        if isinstance(item[6],(bool)):
            if item[6]:
                fnull = 'NULLABLE';
            else:
                fnull = 'NON_NULLABLE';
        else:
            fnull = item[6];

        arcpy.management.AddField(
             in_table          = target
            ,field_name        = item[0]
            ,field_type        = item[1]
            ,field_alias       = item[2]
            ,field_length      = item[3]
            ,field_precision   = item[4]
            ,field_scale       = item[5]
            ,field_is_nullable = fnull
        );

if arcpy.Exists(geopkg) and g_purge:
    arcpy.Delete_management(base + os.sep + 'CANC.gpkg');

if not arcpy.Exists(geopkg):
    
    arcpy.management.CreateSQLiteDatabase(
         out_database_name = base + os.sep + 'CANC.gpkg'
        ,spatial_type      = 'GEOPACKAGE_1.3'
    );
        
    
if not arcpy.Exists(geopkg + os.sep + 'tigerNC'):
    
    arcpy.management.CreateFeatureclass(
         out_path      = geopkg
        ,out_name      = 'tigerNC'
        ,geometry_type = 'POLYGON'
        ,has_m         = 'DISABLED'
        ,has_z         = 'DISABLED'
        ,spatial_reference = arcpy.SpatialReference(4326)
    );
    
    add_fields(
         target = geopkg + os.sep + 'tigerNC'
        ,farray = [
             ['stusab'    ,'TEXT'  ,'USPS State Code'  ,2  ,None,None,False]
        ]
    );
    
    arcpy.management.AddIndex(
         in_table   = geopkg + os.sep + 'tigerNC'
        ,fields     = ['stusab']
        ,index_name = 'STUSAB_PK'
        ,unique     = 'UNIQUE'
    );
    
if not arcpy.Exists(geopkg + os.sep + 'tigerNCbbox'):
    
    arcpy.management.CreateFeatureclass(
         out_path      = geopkg
        ,out_name      = 'tigerNCbbox'
        ,geometry_type = 'POLYGON'
        ,has_m         = 'DISABLED'
        ,has_z         = 'DISABLED'
        ,spatial_reference = arcpy.SpatialReference(4326)
    );
    
    add_fields(
         target = geopkg + os.sep + 'tigerNCbbox'
        ,farray = [
             ['stusab'    ,'TEXT'  ,'USPS State Code'  ,2  ,None,None,False]
        ]
    );
    
    arcpy.management.AddIndex(
         in_table   = geopkg + os.sep + 'tigerNCbbox'
        ,fields     = ['stusab']
        ,index_name = 'STUSAB_PK'
        ,unique     = 'UNIQUE'
    );
    
if not arcpy.Exists(geopkg + os.sep + 'tigerNCbuf25m'):
    
    arcpy.management.CreateFeatureclass(
         out_path      = geopkg
        ,out_name      = 'tigerNCbuf25m'
        ,geometry_type = 'POLYGON'
        ,has_m         = 'DISABLED'
        ,has_z         = 'DISABLED'
        ,spatial_reference = arcpy.SpatialReference(4326)
    );
    
    add_fields(
         target = geopkg + os.sep + 'tigerNCbuf25m'
        ,farray = [
             ['stusab'    ,'TEXT'  ,'USPS State Code'  ,2  ,None,None,False]
        ]
    );
    
    arcpy.management.AddIndex(
         in_table   = geopkg + os.sep + 'tigerNCbuf25m'
        ,fields     = ['stusab']
        ,index_name = 'STUSAB_PK'
        ,unique     = 'UNIQUE'
    );
    
if not arcpy.Exists(geopkg + os.sep + 'tigerNCbuf25mbbox'):
    
    arcpy.management.CreateFeatureclass(
         out_path      = geopkg
        ,out_name      = 'tigerNCbuf25mbbox'
        ,geometry_type = 'POLYGON'
        ,has_m         = 'DISABLED'
        ,has_z         = 'DISABLED'
        ,spatial_reference = arcpy.SpatialReference(4326)
    );
    
    add_fields(
         target = geopkg + os.sep + 'tigerNCbuf25mbbox'
        ,farray = [
             ['stusab'    ,'TEXT'  ,'USPS State Code'  ,2  ,None,None,False]
        ]
    );
    
    arcpy.management.AddIndex(
         in_table   = geopkg + os.sep + 'tigerNCbuf25mbbox'
        ,fields     = ['stusab']
        ,index_name = 'STUSAB_PK'
        ,unique     = 'UNIQUE'
    );
    
with arcpy.da.UpdateCursor(
     in_table    = geopkg + os.sep + 'tigerNC'
    ,field_names = ['stusab','SHAPE@']
) as ucursor:
    
    for row in ucursor:
        ucursor.deleteRow();
        
with arcpy.da.UpdateCursor(
     in_table    = geopkg + os.sep + 'tigerNCbbox'
    ,field_names = ['stusab','SHAPE@']
) as ucursor:
    
    for row in ucursor:
        ucursor.deleteRow();
        
with arcpy.da.UpdateCursor(
     in_table    = geopkg + os.sep + 'tigerNCbuf25m'
    ,field_names = ['stusab','SHAPE@']
) as ucursor:
    
    for row in ucursor:
        ucursor.deleteRow();
        
with arcpy.da.UpdateCursor(
     in_table    = geopkg + os.sep + 'tigerNCbuf25mbbox'
    ,field_names = ['stusab','SHAPE@']
) as ucursor:
    
    for row in ucursor:
        ucursor.deleteRow();
        

In [3]:
tms = MapServiceLayer(
    url = 'https://tigerweb.geo.census.gov/arcgis/rest/services/TIGERweb/State_County/MapServer/0'
);
   
key = 'NC';
print("  fetching geometry for " + key);
f = tms.query(
     where = "STUSAB = '" + key + "' "
    ,outSR = 4326
);
shape = Geometry(f.features[0].geometry).as_arcpy;

print("  inserting " + key + " into tigerNC fc...");
with arcpy.da.InsertCursor(
     in_table    = geopkg + os.sep + 'tigerNC'
    ,field_names = ['stusab','SHAPE@']
) as icursor:
    icursor.insertRow([key,shape]);

print("  generating bbox for NC...");
with arcpy.da.SearchCursor(
     in_table    = geopkg + os.sep + 'tigerNC'
    ,field_names = ['stusab','SHAPE@']
) as scursor:
    
    for row in scursor:
        stusab = row[0]
        shape  = row[1];
    
        with arcpy.da.InsertCursor(
             in_table    = geopkg + os.sep + 'tigerNCbbox'
            ,field_names = ['stusab','SHAPE@']
        ) as icursor:
            
            pt1 = arcpy.Point(shape.extent.XMin,shape.extent.YMin);
            pt2 = arcpy.Point(shape.extent.XMax,shape.extent.YMin);
            pt3 = arcpy.Point(shape.extent.XMax,shape.extent.YMax);
            pt4 = arcpy.Point(shape.extent.XMin,shape.extent.YMax);
            
            bbox = arcpy.Polygon(arcpy.Array([
                 pt1
                ,pt2
                ,pt3
                ,pt4
                ,pt1
            ]));
            
            icursor.insertRow([stusab,bbox]);
            print('');
            print('"bbox":[' + str(shape.extent.XMin) + ',' + str(shape.extent.YMin) + ',' + str(shape.extent.XMax) + ',' + str(shape.extent.YMax) + ']')
            print('');
            
arcpy.env.overwriteOutput = True;
arcpy.analysis.Buffer(
     in_features              = geopkg + os.sep + 'tigerNC'
    ,out_feature_class        = geopkg + os.sep + 'tigerNCbuf25m'
    ,buffer_distance_or_field = '25 mile'
    ,method                   = 'GEODESIC'
);

print("  Generating bbox for buffered state polygon...")
with arcpy.da.SearchCursor(
     in_table    = geopkg + os.sep + 'tigerNCbuf25m'
    ,field_names = ['stusab','SHAPE@']
) as scursor:
    
    for row in scursor:
        stusab = row[0]
        shape  = row[1];
        
        with arcpy.da.InsertCursor(
             in_table    = geopkg + os.sep + 'tigerNCbuf25mbbox'
            ,field_names = ['stusab','SHAPE@']
        ) as icursor:
        
            pt1 = arcpy.Point(shape.extent.XMin,shape.extent.YMin);
            pt2 = arcpy.Point(shape.extent.XMax,shape.extent.YMin);
            pt3 = arcpy.Point(shape.extent.XMax,shape.extent.YMax);
            pt4 = arcpy.Point(shape.extent.XMin,shape.extent.YMax);
            
            bbox = arcpy.Polygon(arcpy.Array([
                 pt1
                ,pt2
                ,pt3
                ,pt4
                ,pt1
            ]));
        
            icursor.insertRow([stusab,bbox]);
            print('');
            print('"bbox":[' + str(shape.extent.XMin) + ',' + str(shape.extent.YMin) + ',' + str(shape.extent.XMax) + ',' + str(shape.extent.YMax) + ']')
            print('');
    

  fetching geometry for NC
  inserting NC into tigerNC fc...
  generating bbox for NC...

"bbox":[-84.32182099999994,33.75287800000007,-75.40011899999996,36.588137000000074]

  Generating bbox for buffered state polygon...

"bbox":[-84.76249142699999,33.39013356500004,-74.95625628499994,36.95068971100005]



In [10]:
gis = GIS();

map1 = gis.map('North Carolina');
df1 = pd.DataFrame.spatial.from_featureclass(geopkg + os.sep + 'tigerNC');
df1.spatial.plot(map_widget = map1);

df2 = pd.DataFrame.spatial.from_featureclass(geopkg + os.sep + 'tigerNCbbox');
df2.spatial.plot(map_widget = map1);

df3 = pd.DataFrame.spatial.from_featureclass(geopkg + os.sep + 'tigerNCbuf25m');
df3.spatial.plot(map_widget = map1);

df4 = pd.DataFrame.spatial.from_featureclass(geopkg + os.sep + 'tigerNCbuf25mbbox');
df4.spatial.plot(map_widget = map1);

map1

MapView(layout=Layout(height='400px', width='100%'))